<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E5%B0%9D%E8%AF%95%E5%8E%BB%E6%8F%90%E5%8D%87C_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken yfinance duckduckgo-search


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import os

os.environ["OPENAI_API_KEY"] = "sk-CYdZPyJN"

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.200
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Understanding OpenAI Functions

### Simple example

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

### Setting up YFinance


In [ ]:
import yfinance as yf

def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'][0], 2)

# use the function
print(get_stock_price('AAPL'))



183.31


In [ ]:
# use the function
print(get_stock_price('GOOG'))

124.43


### Setting up tools

In [3]:
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
# # from langchain.tools import DuckDuckGoSearchTool
# from langchain.agents import Tool
# from langchain.tools import BaseTool

# # search = DuckDuckGoSearchTool()
# # # defining a single tool
# # tools = [
# #     Tool(
# #         name = "search",
# #         func=search.run,
# #         description="useful for when you need to answer questions about current events. You should ask targeted questions"
# #     )
# # ]

In [ ]:
from pydantic import BaseModel, Field

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")


In [ ]:
class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


In [ ]:
# StockPriceTool = Tool(
#     name='Get Stock Ticker price',
#     func= get_stock_price,
#     description="Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"
# )

## LangChain doing it Manualy

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage

In [14]:
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [ ]:
from langchain.tools import MoveFileTool, format_tool_to_openai_function

In [ ]:
{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
                'description': 'Input for MoveFileTool.',
                'type': 'object',
                'properties': {'source_path': {'title': 'Source Path',
                                               'description': 'Path of the file to move',
                                               'type': 'string'},
                               'destination_path': {'title': 'Destination Path',
                                               'description': 'New path for the moved file',
                                               'type': 'string'}},
                'required': ['source_path', 'destination_path']}}

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
  'description': 'Input for MoveFileTool.',
  'type': 'object',
  'properties': {'source_path': {'title': 'Source Path',
    'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'title': 'Destination Path',
    'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [ ]:
# |MoveFileTool().args_schema.schema()

In [ ]:
tools = [StockPriceTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
functions[0]

{'name': 'get_stock_ticker_price',
 'description': 'Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API',
 'parameters': {'title': 'StockPriceCheckInput',
  'description': 'Input for Stock price check.',
  'type': 'object',
  'properties': {'stockticker': {'title': 'Stockticker',
    'description': 'Ticker symbol for stock or index',
    'type': 'string'}},
  'required': ['stockticker']}}

In [ ]:
ai_message = model.predict_messages([HumanMessage(content='What is the price of Google stock')], functions=functions)

In [ ]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_stock_ticker_price', 'arguments': '{\n  "stockticker": "GOOGL"\n}'}}, example=False)

In [ ]:
ai_message.additional_kwargs['function_call']

{'name': 'get_stock_ticker_price',
 'arguments': '{\n  "stockticker": "GOOGL"\n}'}

In [ ]:
import json
_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'stockticker': 'GOOGL'}

In [ ]:
tool_result = tools[0](_args)
tool_result

123.83

In [ ]:
FunctionMessage(name='get_stock_ticker_price',content=tool_result)

FunctionMessage(content='123.83', additional_kwargs={}, name='get_stock_ticker_price')

In [ ]:
final_message = model.predict_messages([HumanMessage(content='What is the price of Google stock'),
                                  ai_message,
                                  FunctionMessage(name='get_stock_ticker_price',content=tool_result),
                                  ], functions=functions)

In [ ]:
final_message

AIMessage(content='The current price of Google stock is $123.83.', additional_kwargs={}, example=False)

## Putting it together as an Agent

In [15]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [5]:
!pip install -q google-search-results

  Preparing metadata (setup.py) ... done


In [24]:
import os
os.environ["SERPAPI_API_KEY"] = "cb9077ae90adcd"

In [26]:
from langchain.agents import load_tools
tools = ["serpapi"]
tools = load_tools(tools, llm=llm)

In [27]:
print(tools[0].description)

A search engine. Useful for when you need to answer questions about current events. Input should be a search query.


In [32]:
tools[0].description = "回答任何问题之前，搜索问题相关的关键信息。"

In [33]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [49]:
# system prompt
open_ai_agent.agent.prompt.messages[0].content = """
You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2021-09
Current date: [2023-06]

Whenever you answer any question, you always first search the key information via search tool.
If the information is still not enough, you can search further more information at most 4 times.
"""

In [50]:
problem = """
  关于DNA和RNA的叙述，正确的是（   ）
  A.DNA有氢键，RNA没有氢键
  B.一种病毒同时含有DNA和RNA
  C.原核细胞中既有DNA，也有RNA
  D.叶绿体、线粒体和核糖体都含有DNA
"""

In [51]:
open_ai_agent.run(problem)



> Entering new  chain...

Invoking: `Search` with `DNA和RNA的特点`


与DNA相似，大部分有生物活性的RNA，包括mRNA、tRNA、rRNA、snRNA，以及其他一些非编码RNA，虽然是单链，但含有自我互补的序列，能使得它们能进行折叠，形成互补双链结构（ ...
Invoking: `Search` with `DNA和RNA的区别`


DNA是遗传物质的载体，也就是说我们细胞核内的染色体就是由DNA主要组成，DNA是由四种碱基构成，ATGC； RNA是负责翻译蛋白质的，DNA在逆转录酶的作用下合成互补的RNA，RNA也是由四种碱基组成，AUGC，也就是说在RNA中U替代了DNA中的T。
Invoking: `Search` with `DNA和RNA的共同点`


RNA的结构与DNA非常相似：每个核苷酸都有一个碱基（环状化合物），一个核糖和一个磷酸基团。 然而，RNA分子是单链的，同时可以具有非常复杂的3D结构。 与DNA相同，RNA有四个核苷酸碱基，其中三个与DNA相同：腺嘌呤，鸟嘌呤和胞嘧啶。
Invoking: `Search` with `DNA和RNA的差异`


DNA是遗传物质的载体，也就是说我们细胞核内的染色体就是由DNA主要组成，DNA是由四种碱基构成，ATGC； RNA是负责翻译蛋白质的，DNA在逆转录酶的作用下合成互补的RNA，RNA也是由四种碱基组成，AUGC，也就是说在RNA中U替代了DNA中的T。根据搜索结果，正确的叙述是：

C. 原核细胞中既有DNA，也有RNA

原核细胞是指没有真核细胞核的细胞，包括细菌和古细菌。在原核细胞中，既存在DNA（染色体）用于遗传信息的存储，也存在RNA用于基因表达和蛋白质合成。因此，C选项是正确的。

其他选项的叙述是不正确的：

A. DNA有氢键，RNA没有氢键
这个叙述是错误的。DNA和RNA都含有氢键，氢键是DNA和RNA分子内部碱基之间的相互作用。

B. 一种病毒同时含有DNA和RNA
这个叙述是错误的。病毒可以是DNA病毒或RNA病毒，但不会同时含有DNA和RNA。

D. 叶绿体、线粒体和核糖体都含有DNA
这个叙述是部分正确的。叶绿体和线粒体是细胞器，它们都含有自己的DNA，用于自身的遗传信息传递和蛋白质合成。而核

'根据搜索结果，正确的叙述是：\n\nC. 原核细胞中既有DNA，也有RNA\n\n原核细胞是指没有真核细胞核的细胞，包括细菌和古细菌。在原核细胞中，既存在DNA（染色体）用于遗传信息的存储，也存在RNA用于基因表达和蛋白质合成。因此，C选项是正确的。\n\n其他选项的叙述是不正确的：\n\nA. DNA有氢键，RNA没有氢键\n这个叙述是错误的。DNA和RNA都含有氢键，氢键是DNA和RNA分子内部碱基之间的相互作用。\n\nB. 一种病毒同时含有DNA和RNA\n这个叙述是错误的。病毒可以是DNA病毒或RNA病毒，但不会同时含有DNA和RNA。\n\nD. 叶绿体、线粒体和核糖体都含有DNA\n这个叙述是部分正确的。叶绿体和线粒体是细胞器，它们都含有自己的DNA，用于自身的遗传信息传递和蛋白质合成。而核糖体是细胞中的蛋白质合成机器，不含有DNA。所以，D选项是不正确的。'

In [53]:
problem = """
国务院决定，自2016年5月1日起，全面推开营业税改增值税试点，并提出“营改增”要确保所有行业税负只减不增，以推动产业转型、结构优化。
这一改革： ①有利于增加财政收入，实现国家职能 ②通过运用财政政策，调节宏观经济 ③有利于避免重复征税，降低生产成本 ④通过调节收入分配，实现社会公平
A.①③
B.②③
C.②④
D.①④
"""

open_ai_agent.run(problem)



> Entering new  chain...
根据提供的信息，营业税改增值税试点的目标是确保所有行业税负只减不增，以推动产业转型和结构优化。这一改革的影响可以从以下几个方面考虑：

1. 有利于增加财政收入，实现国家职能：通过改革，从营业税向增值税的转变可能会增加财政收入，有助于国家履行职能和提供公共服务。

2. 通过运用财政政策，调节宏观经济：改革可以通过调节增值税税率和优惠政策来影响经济运行，实现宏观经济调控的目标。

3. 有利于避免重复征税，降低生产成本：营业税改增值税可以避免在生产和流通环节中的重复征税，减少企业的税负，降低生产成本，提升市场竞争力。

4. 通过调节收入分配，实现社会公平：改革可能通过调节税收政策和税负分配，实现收入的再分配，促进社会公平和经济发展的可持续性。

综上所述，根据提供的选项，正确的答案是C. ②④，即营业税改增值税试点有利于通过运用财政政策调节宏观经济，并通过调节收入分配实现社会公平。

> Finished chain.


'根据提供的信息，营业税改增值税试点的目标是确保所有行业税负只减不增，以推动产业转型和结构优化。这一改革的影响可以从以下几个方面考虑：\n\n1. 有利于增加财政收入，实现国家职能：通过改革，从营业税向增值税的转变可能会增加财政收入，有助于国家履行职能和提供公共服务。\n\n2. 通过运用财政政策，调节宏观经济：改革可以通过调节增值税税率和优惠政策来影响经济运行，实现宏观经济调控的目标。\n\n3. 有利于避免重复征税，降低生产成本：营业税改增值税可以避免在生产和流通环节中的重复征税，减少企业的税负，降低生产成本，提升市场竞争力。\n\n4. 通过调节收入分配，实现社会公平：改革可能通过调节税收政策和税负分配，实现收入的再分配，促进社会公平和经济发展的可持续性。\n\n综上所述，根据提供的选项，正确的答案是C. ②④，即营业税改增值税试点有利于通过运用财政政策调节宏观经济，并通过调节收入分配实现社会公平。'

In [54]:
problem = """
使用位填充方法，以01111110为位首flag，数据为011011111111111111110010，求问传送时要添加几个0____

A.1
B.2
C.3
D.4
"""


open_ai_agent.run(problem)



> Entering new  chain...

Invoking: `Search` with `位填充方法 01111110 011011111111111111110010`


... 'question': '使用位填充方法，以01111110为位首flag，数据为011011111111111111110010，求问传送时要添加几个0____', 'A': '1', 'B': '2', 'C': '3', 'D': '4', ...根据搜索结果，使用位填充方法，以01111110为位首flag，数据为011011111111111111110010时，传送时需要添加2个0。因此答案是B. 2。

> Finished chain.


'根据搜索结果，使用位填充方法，以01111110为位首flag，数据为011011111111111111110010时，传送时需要添加2个0。因此答案是B. 2。'

In [55]:
from langchain.chains import PALChain
pal_chain = PALChain.from_math_prompt(llm, verbose=True)


In [59]:
response = pal_chain.run("Using bit-stuffing method with 01111110 as the head flag, and data as 011011111111111111110010, how many 0's need to be added during transmission?")



> Entering new  chain...
def solution():
    """Using bit-stuffing method with 01111110 as the head flag, and data as 011011111111111111110010, how many 0's need to be added during transmission?"""
    data = "011011111111111111110010"
    head_flag = "01111110"
    stuffed_data = head_flag + data.replace("11111", "111110") + head_flag
    num_zeros_added = stuffed_data.count("0") - data.count("0")
    result = num_zeros_added
    return result

> Finished chain.


In [60]:
print( response)

7


## OpenAI Functions Agent

### Advantages
- Better reasoning and tool selection than ReACT / Toolformer?
- Less tokens needed

### Disadvantages
- Not as easy to customize via prompt change etc if it doesn't work
- locks you code into the OpenAI way
- Still need tokens for Tool/Function descriptions

## Multiple Tools

In [ ]:
from datetime import datetime, timedelta

def get_price_change_percent(symbol, days_ago):
    ticker = yf.Ticker(symbol)

    # Get today's date
    end_date = datetime.now()

    # Get the date N days ago
    start_date = end_date - timedelta(days=days_ago)

    # Convert dates to string format that yfinance can accept
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    # Get the historical data
    historical_data = ticker.history(start=start_date, end=end_date)

    # Get the closing price N days ago and today's closing price
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    # Calculate the percentage change
    percent_change = ((new_price - old_price) / old_price) * 100

    return round(percent_change, 2)

# Use the function
print(get_price_change_percent('AAPL', 30))  # for 30 days ago


6.53


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)

def get_best_performing(stocks, days_ago):
    best_stock = None
    best_performance = None
    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            print(f"Could not calculate performance for {stock}: {e}")
    return best_stock, best_performance

stocks = ['AAPL', 'MSFT', 'GOOG']
days_ago = 90  # change as desired

best_stock, best_performance = get_best_performing(stocks, days_ago)
print(f"The best performing stock over the past {days_ago} days is {best_stock} with a performance of {best_performance}%")


The best performing stock over the past 90 days is GOOG with a performance of 23.11%


### Make the Tools

In [ ]:
from typing import List


class StockChangePercentageCheckInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockPercentageChangeTool(BaseTool):
    name = "get_price_change_percent"
    description = "Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stockticker: str, days_ago: int):
        price_change_response = get_price_change_percent(stockticker, days_ago)

        return price_change_response

    def _arun(self, stockticker: str, days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput


# the best performing

class StockBestPerformingInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stocktickers: List[str] = Field(..., description="Ticker symbols for stocks or indices")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockGetBestPerformingTool(BaseTool):
    name = "get_best_performing"
    description = "Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stocktickers: List[str], days_ago: int):
        price_change_response = get_best_performing(stocktickers, days_ago)

        return price_change_response

    def _arun(self, stockticker: List[str], days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput

In [ ]:
tools = [StockPriceTool(),StockPercentageChangeTool(), StockGetBestPerformingTool()]

# functions = [format_tool_to_openai_function(t) for t in tools]

In [ ]:
tools

[StockPriceTool(name='get_stock_ticker_price', description='Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API', args_schema=<class '__main__.StockPriceCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockPercentageChangeTool(name='get_price_change_percent', description="Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over", args_schema=<class '__main__.StockChangePercentageCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockGetBestPerformingTool(name='get_best_performing', description='Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [ ]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [ ]:
open_ai_agent.run("What is the price of Google stock today?")



> Entering new  chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


123.83The price of Google stock today is $123.83.

> Finished chain.


'The price of Google stock today is $123.83.'

In [ ]:
open_ai_agent.run("Has google's stock gone up over the past 90 days?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


23.44Yes, Google's stock (GOOGL) has gone up by 23.44% over the past 90 days.

> Finished chain.


"Yes, Google's stock (GOOGL) has gone up by 23.44% over the past 90 days."

In [ ]:
open_ai_agent.run("How much has google's stock gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


23.44Google's stock has gone up by 23.44% over the past 3 months.

> Finished chain.


"Google's stock has gone up by 23.44% over the past 3 months."

In [ ]:
open_ai_agent.run("Which stock out of Google, Meta and MSFT has performed best over the past 3 months?")



> Entering new  chain...

Invoking: `get_best_performing` with `{'stocktickers': ['GOOGL', 'META', 'MSFT'], 'days_ago': 90}`


('META', 32.4)The stock that has performed the best over the past 3 months out of Google, Meta, and MSFT is Meta, with a return of 32.4%.

> Finished chain.


'The stock that has performed the best over the past 3 months out of Google, Meta, and MSFT is Meta, with a return of 32.4%.'

In [ ]:
open_ai_agent.run("How much has MSFT's stock gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'MSFT', 'days_ago': 90}`


21.3Microsoft's stock (MSFT) has gone up by 21.3% over the past 3 months.

> Finished chain.


"Microsoft's stock (MSFT) has gone up by 21.3% over the past 3 months."

In [ ]:
open_ai_agent.run("How much has Bitcoin gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'BTC-USD', 'days_ago': 90}`


3.46Bitcoin has gone up by 3.46% over the past 3 months.

> Finished chain.


'Bitcoin has gone up by 3.46% over the past 3 months.'